**1. 데이터 불러오기**

In [1]:
# 데이터 불러오기
import pandas as pd
original_data = pd.read_csv('./data/total_price.csv')

# 불필요한 열 제거
del original_data['Unnamed: 0']

# 날짜를 인덱스로 변경
original_data = original_data.set_index('date')

# l1 ~ l4 Line Setting
original_data['l1'] = original_data['l1'].apply(lambda x : str(x)).apply(lambda x : x[1:])
original_data['l2'] = original_data['l2'].apply(lambda x : str(x)).apply(lambda x : x[1:])
original_data['l3'] = original_data['l3'].apply(lambda x : str(x)).apply(lambda x : x[1:])
original_data['l4'] = original_data['l4'].apply(lambda x : str(x)).apply(lambda x : x[1:])

# Data Head
original_data

,open,high,low,close,trading_volume,score,index,probability,l1,l2,l3,l4,lgap,lrate,code
date,,,,,,,,,,,,,,,
20160622,10150,10150,9780,9830,315346,2.586,0.538,2.477,10175.0,9990.0,9805.0,9620.0,555.0,6,20
20160623,9710,9870,9510,9730,293348,2.778,1.429,47.114,9980.0,9800.0,9620.0,9440.0,540.0,6,20
20160624,9840,9910,8700,9080,621895,4.162,0.960,27.960,10100.0,9495.0,8890.0,8285.0,1815.0,20,20
20160627,8750,9480,8750,9400,334886,3.940,0.880,23.643,9805.0,9440.0,9075.0,8710.0,1095.0,12,20
20160628,9210,9770,9210,9760,282254,3.940,0.880,23.643,10045.0,9765.0,9485.0,9205.0,840.0,9,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20200219,14750,14880,14750,14850,1090,2857.000,10.000,79.010,14930.0,14865.0,14800.0,14735.0,195.0,1,241180
20200220,14850,15005,14790,14850,3900,2732.000,20.000,79.751,15035.0,14927.5,14820.0,14712.5,322.5,2,241180
20200221,14695,14845,14695,14845,1706,2722.000,20.000,79.751,14920.0,14845.0,14770.0,14695.0,225.0,2,241180


**2. 데이터 전처리**

In [2]:
# 라이브러리
import numpy as np
import pandas as pd

# 종목 리스트
codes = original_data.code.unique()

# 최종 데이터프레임을 저장할 공간
dataset = pd.DataFrame()

# 중간 결과 집계
COUNT = 0

# 종목별 전처리 진행
for code in codes : 
    
    # 날짜별로 데이터 재정렬
    data = original_data[original_data['code'] == code].reset_index().sort_values(by=['date'])
    
    # open 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['open'] = 100 * (data['open']-data['open'].shift(1)) / data['open'].shift(1)
    data['open'] = data['open'].apply(lambda x : np.float(x))
    data['open'] = data['open'].apply(lambda x : round(x,2))
    
    # high 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['high'] = 100*(data['high']-data['high'].shift(1)) / data['high'].shift(1)
    data['high'] = data['high'].apply(lambda x : np.float(x))
    data['high'] = data['high'].apply(lambda x : round(x,2))

    # low 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['low']=100*(data['low']-data['low'].shift(1))/data['low'].shift(1)
    data['low']=data['low'].apply(lambda x : np.float(x))
    data['low']=data['low'].apply(lambda x : round(x,2))

    # close 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['close']=100*(data['close']-data['close'].shift(1))/data['close'].shift(1)
    data['close']=data['close'].apply(lambda x : np.float(x))
    data['close']=data['close'].apply(lambda x : round(x,2))

    # trading_volume 실수화한 다음, 반올림 (로그변환 하기 전에 +1 해줌)
    data['trading_volume']=data['trading_volume'].apply(lambda x : np.float(x))
    data['trading_volume']=data['trading_volume'].apply(lambda x : np.log(x + 1))
    data['trading_volume']=data['trading_volume'].apply(lambda x : round(x,2))

    # Score는 실수화하고, 로그변환만 진행하고 반올림 (로그변환 하기 전에 +1 해줌)
    data['score']=data['score'].apply(lambda x : np.float(x))
    data['score']=data['score'].apply(lambda x : np.log(x +1))
    data['score']=data['score'].apply(lambda x : round(x,2))

    # index는 실수화하고, 로그변환만 진행하고 반올림 (로그변환 하기 전에 +1 해줌)
    data['index']=data['index'].apply(lambda x : np.float(x))
    data['index']=data['index'].apply(lambda x : np.log(x + 1))
    data['index']=data['index'].apply(lambda x : round(x,2))
    
    # probability는 그대로 이용하고 반올림
    data['probability']=data['probability'].apply(lambda x : np.float(x))
    data['probability']=data['probability'].apply(lambda x : round(x,2))

    # l1 line 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['l1']=data['l1'].apply(lambda x : np.float(x))
    data['l1']=100*(data['l1']-data['l1'].shift(1))/data['l1'].shift(1)
    data['l1']=data['l1'].apply(lambda x : np.float(x))
    data['l1']=data['l1'].apply(lambda x : round(x,2))
    
    # l2 line 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['l2']=data['l2'].apply(lambda x : np.float(x))
    data['l2']=100*(data['l2']-data['l2'].shift(1))/data['l2'].shift(1)
    data['l2']=data['l2'].apply(lambda x : np.float(x))
    data['l2']=data['l2'].apply(lambda x : round(x,2))

    # l3 line 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['l3']=data['l3'].apply(lambda x : np.float(x))
    data['l3']=100*(data['l3']-data['l3'].shift(1))/data['l3'].shift(1)
    data['l3']=data['l3'].apply(lambda x : np.float(x))
    data['l3']=data['l3'].apply(lambda x : round(x,2))

    # l1 line 가격을 전일대비 변동폭으로 계산하고, 실수화한 다음, 반올림
    data['l4']=data['l4'].apply(lambda x : np.float(x))
    data['l4']=100*(data['l4']-data['l4'].shift(1))/data['l4'].shift(1)
    data['l4']=data['l4'].apply(lambda x : np.float(x))
    data['l4']=data['l4'].apply(lambda x : round(x,2))

    # lgap는 로그변환만하고, 반올림 (로그변환 하기 전에 +1 해줌)
    data['lgap']=data['lgap'].apply(lambda x : np.float(x))
    data['lgap']=data['lgap'].apply(lambda x : np.log(x + 1))
    data['lgap']=data['lgap'].apply(lambda x : round(x,2))

    # lrate는 그대로 이용
    data['lrate']=data['lrate'].apply(lambda x : np.float(x))
    
    # Zone 지정
    data['zone']=100*(data['close']-data['close'].shift(-1))/data['close'].shift(-1)
    
    def zone(x) : 
        if x < -1 : return "Danger"
        elif x < 1 : return "Not Bad"
        else : return "Good"
            
    data['zone'] = data['zone'].apply(lambda x : zone(x))
    
    # 모델에 불필요한 변수 제거
    del data['date']
    del data['code']
    
    # 전처리 과정에서 생긴 결측값 제거
    data = data.dropna()
    
    # 최종 데이터셋에 추가
    dataset = dataset.append(data)
    
    # data 리셋
    data = pd.DataFrame()
    
    # 중간결과 출력
    COUNT += 1
    if COUNT % 100 == 0 : 
        print('{}번째 종목 전처리 진행중.....' .format(COUNT))
        
# 이상값 제거 (전일대비 OHLC가 35% 이상 뛴 종목)
dataset = dataset[dataset['open'] < 35.00]
dataset = dataset[dataset['high'] < 35.00]
dataset = dataset[dataset['low'] < 35.00]
dataset = dataset[dataset['close'] < 35.00]

100번째 종목 전처리 진행중.....
200번째 종목 전처리 진행중.....
300번째 종목 전처리 진행중.....
400번째 종목 전처리 진행중.....
500번째 종목 전처리 진행중.....
600번째 종목 전처리 진행중.....
700번째 종목 전처리 진행중.....
800번째 종목 전처리 진행중.....
900번째 종목 전처리 진행중.....
1000번째 종목 전처리 진행중.....
1100번째 종목 전처리 진행중.....
1200번째 종목 전처리 진행중.....
1300번째 종목 전처리 진행중.....
1400번째 종목 전처리 진행중.....
1500번째 종목 전처리 진행중.....
1600번째 종목 전처리 진행중.....
1700번째 종목 전처리 진행중.....
1800번째 종목 전처리 진행중.....
1900번째 종목 전처리 진행중.....
2000번째 종목 전처리 진행중.....
2100번째 종목 전처리 진행중.....


In [3]:
# 데이터 전처리 결과
dataset

,open,high,low,close,trading_volume,score,index,probability,l1,l2,l3,l4,lgap,lrate,zone
1,-4.33,-2.76,-2.76,-1.02,12.59,1.33,0.89,47.11,-1.92,-1.90,-1.89,-1.87,6.29,6.0,Danger
2,1.34,0.41,-8.52,-6.68,13.34,1.64,0.67,27.96,1.20,-3.11,-7.59,-12.24,7.50,20.0,Danger
3,-11.08,-4.34,0.57,3.52,12.72,1.60,0.63,23.64,-2.92,-0.58,2.08,5.13,7.00,12.0,Danger
4,5.26,3.06,5.26,3.83,12.55,1.60,0.63,23.64,2.45,3.44,4.52,5.68,6.73,9.0,Danger
5,6.95,3.38,5.32,-0.10,12.77,1.60,0.63,23.64,0.80,1.64,2.53,3.48,6.40,6.0,Danger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
897,-0.37,-0.47,-0.17,0.34,6.99,7.96,2.40,79.01,-0.22,-0.07,0.08,0.24,5.28,1.0,Good
898,0.68,0.84,0.27,0.00,8.27,7.91,3.04,79.75,0.70,0.42,0.14,-0.15,5.78,2.0,Danger
899,-1.04,-1.07,-0.64,-0.03,7.44,7.91,3.04,79.75,-0.76,-0.55,-0.34,-0.12,5.42,2.0,Danger
900,-0.31,-1.11,-0.92,-1.48,6.19,7.90,1.95,77.30,-1.39,-1.30,-1.20,-1.11,5.20,1.0,Danger


In [6]:
# 데이터 전처리 결과 요약
round(dataset.describe(), 2)

             open        high         low       close  trading_volume  \
count  1901799.00  1901799.00  1901799.00  1901799.00      1901799.00   
mean        -0.00        0.01       -0.01        0.01           10.79   
std          3.16        3.34        2.64        2.95            2.79   
min        -94.54      -93.68      -95.92      -95.92            0.00   
25%         -1.34       -1.16       -0.98       -1.16            9.69   
50%          0.00       -0.11        0.00        0.00           11.19   
75%          1.17        0.79        0.94        0.96           12.47   
max         34.99       32.48       34.89       32.23           20.55   

            score       index  probability          l1          l2  \
count  1901799.00  1901799.00   1901799.00  1901799.00  1901799.00   
mean         3.28        1.47        52.50        0.03        0.00   
std          3.18        1.08        26.68        3.83        2.86   
min          0.00        0.01         0.00      -93.68      -9

In [7]:
# 구역(zone)별 데이터 전처리 결과 요약 
print(pd.pivot_table(dataset, index = ['zone'],\
                     values = ['open','high','low','close'], aggfunc = 'mean'), end='\n\n\n')
print(pd.pivot_table(dataset, index = ['zone'],\
                     values = ['trading_volume','score','index','probability'], aggfunc = 'mean'), end='\n\n\n')
print(pd.pivot_table(dataset, index = ['zone'],\
                     values = ['l1','l2','l3','l4'], aggfunc = 'mean'), end='\n\n\n')
print(pd.pivot_table(dataset, index = ['zone'],\
                     values = ['lgap','lrate'], aggfunc = 'mean'))

            close      high       low      open
zone                                           
Danger  -0.068325 -0.108825 -0.001233 -0.019990
Good     0.224842  0.376827 -0.058738  0.042865
Not Bad  0.139351 -0.053531  0.206523  0.088034


            index  probability     score  trading_volume
zone                                                    
Danger   1.481675    52.681839  3.306503       10.940185
Good     1.455403    52.025443  3.222047       10.377473
Not Bad  1.433832    51.334126  3.279029       10.006155


               l1        l2        l3        l4
zone                                           
Danger  -0.129220 -0.097713 -0.043091  0.042324
Good     0.520694  0.292166  0.075905 -0.120635
Not Bad -0.080888  0.037507  0.169780  0.319393


             lgap     lrate
zone                       
Danger   5.815741  5.443960
Good     5.562543  5.998649
Not Bad  5.356372  3.638601


![figure](./figure/figure01.png)

![figure](./figure/figure02.png)

![figure](./figure/figure03.png)

![figure](./figure/figure04.png)